In [11]:
import pandas as pd
df_1 = pd.read_csv('../data/4.processed_data/Query4_results_test_processed.csv')
df_2 = pd.read_csv('../data/4.processed_data/Query1503_results_conv_processed.csv')
df_3 = pd.read_csv('../data/4.processed_data/Query1603_results_test_processed.csv')
df_train = pd.concat([df_1,df_2,df_3])
df_test = pd.read_csv('../data/4.processed_data/Query1803_results_processed.csv')

In [12]:
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
mean_savings_Journey_ID_train = df_train.groupby(['Journey_ID','Detected_Country'])['rel_diff_to_min_price_FlightID'].mean().reset_index(name='mean_savings_for_Journey_ID_country')
# Merge the average_savings DataFrame back into df_reduced
# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
median_savings_Journey_route_train = df_train.groupby(['Journey_route','Detected_Country'])['rel_diff_to_min_price_FlightID'].median().reset_index(name='median_savings_for_Journey_route_country')
# Merge the average_savings DataFrame back into df_reduced



# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
mean_savings_Journey_ID_test = df_test.groupby(['Journey_ID','Detected_Country'])['rel_diff_to_min_price_FlightID'].mean().reset_index(name='mean_savings_for_Journey_ID_country')
# Merge the average_savings DataFrame back into df_reduced

# Calculate the average of rel_diff_to_min_price_FlightID for each Journey_ID and Detected_Country
median_savings_Journey_route_test = df_test.groupby(['Journey_route','Detected_Country'])['rel_diff_to_min_price_FlightID'].median().reset_index(name='median_savings_for_Journey_route_country')
# Merge the average_savings DataFrame back into df_reduced

In [13]:
# Step 1: Create a copy of df_reduced to work with
df_enriched = df_train.copy()

# Assuming mean_savings_Journey_ID and median_savings_Journey_route are as previously defined

# Step 2: Merge the mean_savings_for_Journey_ID_country DataFrame into df_enriched
df_enriched = df_enriched.merge(mean_savings_Journey_ID_train, on=['Journey_ID', 'Detected_Country'], how='left')

# Step 3: Merge the median_savings_Journey_route into df_enriched
# Note: Rename 'median_savings_for_Journey_ID_country' to 'median_savings_for_Journey_route' to avoid confusion
df_enriched = df_enriched.merge(median_savings_Journey_route_train, on=['Journey_route', 'Detected_Country'], how='left')

# Step 4: Calculate the simple mean of the two savings values
df_enriched['mean_savings'] = df_enriched[['mean_savings_for_Journey_ID_country', 'median_savings_for_Journey_route_country']].mean(axis=1)

In [ ]:
df_enriched.mean

In [14]:

# Step 2: Merge the mean_savings_for_Journey_ID_country DataFrame into df_enriched
df_test = df_test.merge(mean_savings_Journey_ID_test, on=['Journey_ID', 'Detected_Country'], how='left')

# Step 3: Merge the median_savings_Journey_route into df_enriched
# Note: Rename 'median_savings_for_Journey_ID_country' to 'median_savings_for_Journey_route' to avoid confusion
df_test = df_test.merge(median_savings_Journey_route_test, on=['Journey_route', 'Detected_Country'], how='left')

# Step 4: Calculate the simple mean of the two savings values
df_test['mean_savings'] = df_test[['mean_savings_for_Journey_ID_country', 'median_savings_for_Journey_route_country']].mean(axis=1)


In [15]:
df_test.mean_savings.describe()

count    704.000000
mean       2.022184
std        5.680986
min        0.000000
25%        0.524553
50%        0.767727
75%        1.297744
max       39.028601
Name: mean_savings, dtype: float64

In [16]:
df_enriched.mean_savings.describe()

count    5806.000000
mean        2.630391
std         4.413457
min         0.000000
25%         0.751915
50%         1.981605
75%         3.716443
max       107.490743
Name: mean_savings, dtype: float64

In [17]:
import numpy as np

# Ensure that all values in mean_savings are positive
# You might add a constant to the entire series if there are non-positive values
df_enriched['mean_savings_adjusted'] = df_enriched['mean_savings'] + 1  # Example adjustment

# Apply log transformation
df_enriched['log_mean_savings'] = np.log(df_enriched['mean_savings_adjusted'])

# Similar adjustment and transformation for the test set
df_test['mean_savings_adjusted'] = df_test['mean_savings'] + 1  # Example adjustment
df_test['log_mean_savings'] = np.log(df_test['mean_savings_adjusted'])


In [18]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

X_train = pd.get_dummies(df_enriched[['Journey_route','days_until_departure', 'Detected_Country']])
y_train = df_enriched['mean_savings_adjusted']
feature_names = X_train.columns.tolist()



# Specify your model parameters here
params_grid = {'colsample_bytree': 0.9, 'eta': 0.299, 'max_depth': 7, 'subsample': 0.8}
params_optuna = {'max_depth': 7, 'eta': 0.319923163390001, 'subsample': 0.8, 'colsample_bytree': 0.9}
params_optuna200 = {'max_depth': 8, 'eta': 0.34991201767803204, 'subsample': 0.9, 'colsample_bytree': 0.9}


params_self = {'max_depth': 3, 'eta': 0.01, 'subsample': 0.9, 'colsample_bytree': 0.9}




# Create XGBoost model
model_v8_combined  = xgb.XGBRegressor(**params_optuna200)

# Fit the model on your training data
model_v8_combined.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.34991201767803204,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [19]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
# Assuming new_test_df is your new test data


# Update X_test and y_test accordingly
X_test = pd.get_dummies(df_test[['Journey_route', 'days_until_departure', 'Detected_Country']])
y_test = df_test['mean_savings_adjusted']


# Align the new test set features with the training features
# Add missing columns in the new test set
missing_cols = set(feature_names) - set(X_test.columns)
for c in missing_cols:
    X_test[c] = 0

# Remove any columns in the new test set that are not present in the training set
X_test = X_test[feature_names]

# Now, you can predict with your model
y_pred = model_v8_combined.predict(X_test)



# Calculate and print the Mean Squared Error
mea = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mea}")


r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Mean Absolute Error: 1.9492501323844216
R-squared: 0.6273410855831789


In [11]:
y_diff = y_test - y_pred

In [12]:
y_diff

0     -1.750132
1     -5.488392
2     -2.409301
3     -4.800202
4     -0.883511
         ...   
699   -0.039135
700    0.053587
701    0.020759
702    0.008827
703    0.046731
Name: mean_savings, Length: 704, dtype: float64

In [9]:
df_enriched.Journey_route.nunique()

38

In [10]:
df_enriched.Detected_Country.nunique()

15